**Universidad Internacional de La Rioja (UNIR) - Máster Universitario en Inteligencia Artificial - Procesamiento del Lenguaje Natural**

***
Datos del alumno (Nombre y Apellidos): Jose Manuel Pinillos Rubio

Fecha: 31 de enero de 2025
***

# <span style="font-size: 20pt; font-weight: bold; color: #0098cd;">Laboratorio: *word embeddings* y *transformers* para clasificación de texto</span>

**Objetivos**

Con este laboratorio el alumno comparará diferentes modelos de clasificación de texto mediante el uso de técnicas basadas en word embedings y transformers. El alumno, por tanto, adquirirá dos competencias: primero, la capacidad de aplicar un modelo neuronal para la clasificación de texto y, segundo, la capacidad de comparar diferentes modelos entre sí.

El objetivo es entender los conceptos que se trabajan y ser capaz de hacer pequeñas experimentaciones para mejorar el Notebook creado.

**Descripción**

En esta actividad vamos a trabajar en clasificar textos. Se recorrerá todo el proceso desde traer el dataset hasta proceder a dicha clasificación. Durante la actividad se llevarán a cabo muchos procesos como la creación de un vocabulario, el uso de embeddings y la creación de modelos.

Las cuestiones presentes en esta actividad están basadas en un Notebook creado por François Chollet, uno de los creadores de Keras y autor del libro "Deep Learning with Python".

En este Notebook se trabaja con el dataset "Newsgroup20" que contiene aproximadamente 20000 mensajes que pertenecen a 20 categorías diferentes.

# Librerías

In [1]:
# Importamos las librerías necesarias

import numpy as np  # Biblioteca para manejo de arreglos numéricos y operaciones matemáticas
import tensorflow as tf  # Biblioteca para la construcción y entrenamiento de modelos de aprendizaje profundo
from tensorflow import keras  # Módulo de Keras dentro de TensorFlow para facilitar la implementación de redes neuronales

# Descarga de Datos

In [2]:
# Descargamos el dataset "20 Newsgroups" desde la URL proporcionada
# y lo extraemos automáticamente (untar=True).

data_path = keras.utils.get_file(
    "news20.tar.gz",  # Nombre del archivo que se descargará y almacenará en la caché de Keras
    "http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz",  # URL del dataset
    untar=True,  # Indica que el archivo se descomprimirá automáticamente tras la descarga
)

# 'data_path' almacena la ruta donde se ha descargado y extraído el dataset.
# Para verificar la ruta exacta, la imprimimos en pantalla.
print("El dataset ha sido descargado y extraído en:", data_path)

17329808/17329808 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
El dataset ha sido descargado y extraído en: /root/.keras/datasets/news20_extracted


In [3]:
# Importamos las librerías necesarias para manejar rutas y directorios
import os  # Biblioteca para interactuar con el sistema de archivos
import pathlib  # Biblioteca para manejar rutas de archivos de manera más estructurada

# Definimos la ruta del directorio base donde se encuentra el dataset
data_dir = pathlib.Path(data_path).parent / "news20_extracted" / "20_newsgroup"

# Listamos los nombres de los subdirectorios dentro del dataset
dirnames = os.listdir(data_dir)

# Imprimimos la cantidad de subdirectorios (categorías de noticias) y sus nombres
print("Number of directories:", len(dirnames))  # Muestra cuántas categorías hay en el dataset
display("Directory names:", dirnames)  # Lista los nombres de las categorías de noticias

Number of directories: 20


'Directory names:'

['talk.politics.guns',
 'rec.sport.baseball',
 'comp.windows.x',
 'sci.space',
 'talk.politics.mideast',
 'comp.sys.mac.hardware',
 'talk.politics.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.graphics',
 'misc.forsale',
 'talk.religion.misc',
 'alt.atheism',
 'sci.electronics',
 'rec.motorcycles',
 'sci.crypt',
 'sci.med',
 'rec.sport.hockey',
 'soc.religion.christian',
 'comp.os.ms-windows.misc',
 'rec.autos']

In [4]:
# Imprimimos la ruta del directorio donde se encuentra el dataset después de la extracción
print("Ruta del dataset:", data_dir)

Ruta del dataset: /root/.keras/datasets/news20_extracted/20_newsgroup


In [5]:
# Listamos los archivos dentro de la categoría "comp.graphics", una de las categorías del dataset "20 Newsgroups"
fnames = os.listdir(data_dir / "comp.graphics")  # Obtenemos la lista de archivos en la carpeta "comp.graphics"

# Imprimimos el número total de archivos en esta categoría
print("Number of files in comp.graphics:", len(fnames))

# Mostramos los nombres de los primeros 5 archivos como ejemplo
print("Some example filenames:", fnames[:5])

Number of files in comp.graphics: 1000
Some example filenames: ['38591', '38357', '38235', '38794', '38899']


In [6]:
#Ejemplo de un texto de la categoría "com.graphics"
print(open(data_dir / "comp.graphics" / "37261").read())

Xref: cantaloupe.srv.cs.cmu.edu comp.graphics:37261 alt.graphics:519 comp.graphics.animation:2614
Path: cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!ogicse!uwm.edu!zaphod.mps.ohio-state.edu!darwin.sura.net!dtix.dt.navy.mil!oasys!lipman
From: lipman@oasys.dt.navy.mil (Robert Lipman)
Newsgroups: comp.graphics,alt.graphics,comp.graphics.animation
Subject: CALL FOR PRESENTATIONS: Navy SciViz/VR Seminar
Message-ID: <32850@oasys.dt.navy.mil>
Date: 19 Mar 93 20:10:23 GMT
Article-I.D.: oasys.32850
Expires: 30 Apr 93 04:00:00 GMT
Reply-To: lipman@oasys.dt.navy.mil (Robert Lipman)
Followup-To: comp.graphics
Distribution: usa
Organization: Carderock Division, NSWC, Bethesda, MD
Lines: 65


			CALL FOR PRESENTATIONS
	
      NAVY SCIENTIFIC VISUALIZATION AND VIRTUAL REALITY SEMINAR

			Tuesday, June 22, 1993

	    Carderock Division, Naval Surface Warfare Center
	      (formerly the David Taylor Research Center)
			  Bethesda, Maryland

SPONSOR: NESS (Navy Engineering Software System) is sponsori

---

## Pregunta 1

### 1.1 - Utilizando el tokenizador de spacy, que ya conoces, calcula el número promedio de tokens de una muestra de 15 ficheros de la categoría «com.graphics». Indica el código utilizado y el resultado obtenido.

In [7]:
# Importamos la biblioteca spaCy para el procesamiento del lenguaje natural
import spacy
import en_core_web_sm  # Cargamos el modelo de spaCy para el idioma inglés

# Cargamos el modelo de procesamiento de texto en inglés "en_core_web_sm"
nlp = en_core_web_sm.load()

# Inicializamos una variable para contar el número total de tokens
total_tokens = 0

# Aseguramos que tomamos los primeros 15 archivos
num_files = 15
fnames_subset = fnames[:num_files]  # Tomamos los primeros 15 archivos de la lista

# Iteramos sobre los primeros 15 archivos de la categoría "comp.graphics"
for fname in fnames_subset:
    # Leemos el contenido del archivo y lo procesamos con spaCy
    doc = nlp(pathlib.Path(data_dir / "comp.graphics" / fname).read_text(encoding="latin-1"))

    # Obtenemos la longitud del documento en tokens y la sumamos al total
    total_tokens += len(doc)

# Calculamos el número promedio de tokens en los 15 archivos analizados
average_tokens = total_tokens / num_files

# Imprimimos el resultado
print(f"Promedio de tokens en los primeros 15 documentos de 'comp.graphics': {average_tokens:.2f}")

Promedio de tokens en los primeros 15 documentos de 'comp.graphics': 281.53


Este código calcula el número promedio de *tokens* en 15 archivos de la categoría *comp.graphics* utilizando *spaCy* para la tokenización. Para ello, primero se inicializa la variable `total_tokens` en cero, que servirá para almacenar la cantidad total de *tokens* en todos los archivos procesados.

Se define la variable `num_files` con el valor 15 para indicar cuántos archivos se van a analizar. Luego, se obtiene una lista con los nombres de los primeros 15 archivos dentro de la carpeta *comp.graphics* utilizando `fnames[:num_files]`, lo que garantiza que solo se procesen los archivos necesarios.

A continuación, se inicia un bucle que recorre cada uno de estos archivos. Para cada archivo, se construye su ruta completa utilizando `pathlib.Path(data_dir / "comp.graphics" / fname)`, asegurando así que se accede correctamente a su contenido. La función `read_text(encoding="latin-1")` se encarga de leer el contenido del archivo como texto, utilizando la codificación *latin-1* para manejar caracteres especiales sin errores. Una vez obtenido el texto del archivo, se pasa a la función `nlp()` de *spaCy*, que tokeniza el texto y devuelve un objeto `doc` que contiene los *tokens* generados.

La cantidad de *tokens* en el documento se obtiene con `len(doc)`, que devuelve el número total de *tokens* en el texto procesado. Este valor se suma a la variable `total_tokens`, acumulando así el número total de *tokens* de todos los archivos analizados.

Después de recorrer los 15 archivos y sumar la cantidad total de *tokens*, se calcula el promedio dividiendo `total_tokens` entre `num_files`. Finalmente, el resultado se imprime en pantalla con un mensaje claro, utilizando una *f-string* para formatear la salida y asegurando que el promedio de *tokens* se muestre con solo dos decimales mediante `{average_tokens:.2f}`.

El resultado obtenido indica que, en promedio, cada uno de los primeros 15 documentos de la categoría *comp.graphics* contiene aproximadamente **276.13 tokens**. Esto refleja el tamaño medio de los textos en esta categoría después del proceso de tokenización con *spaCy*.

---

In [8]:
# Listamos los archivos dentro de la categoría "talk.politics.misc", otra de las categorías del dataset "20 Newsgroups"

fnames = os.listdir(data_dir / "talk.politics.misc")  # Obtenemos la lista de archivos en la carpeta "talk.politics.misc"

# Imprimimos el número total de archivos en esta categoría
print("Number of files in talk.politics.misc:", len(fnames))

# Mostramos los nombres de los primeros 5 archivos como ejemplo
print("Some example filenames:", fnames[:5])

Number of files in talk.politics.misc: 1000
Some example filenames: ['178867', '178722', '178818', '178816', '176929']


In [9]:
#Ejemplo de un texto de la categoría "talk.politics.misc"
print(open(data_dir / "talk.politics.misc" / "178463").read())

Xref: cantaloupe.srv.cs.cmu.edu talk.politics.guns:54219 talk.politics.misc:178463
Newsgroups: talk.politics.guns,talk.politics.misc
Path: cantaloupe.srv.cs.cmu.edu!magnesium.club.cc.cmu.edu!news.sei.cmu.edu!cis.ohio-state.edu!magnus.acs.ohio-state.edu!usenet.ins.cwru.edu!agate!spool.mu.edu!darwin.sura.net!martha.utcc.utk.edu!FRANKENSTEIN.CE.UTK.EDU!VEAL
From: VEAL@utkvm1.utk.edu (David Veal)
Subject: Re: Proof of the Viability of Gun Control
Message-ID: <VEAL.749.735192116@utkvm1.utk.edu>
Lines: 21
Sender: usenet@martha.utcc.utk.edu (USENET News System)
Organization: University of Tennessee Division of Continuing Education
References: <1qpbqd$ntl@access.digex.net> <C5otvp.ItL@magpie.linknet.com>
Date: Mon, 19 Apr 1993 04:01:56 GMT

[alt.drugs and alt.conspiracy removed from newsgroups line.]

In article <C5otvp.ItL@magpie.linknet.com> neal@magpie.linknet.com (Neal) writes:

>   Once the National Guard has been called into federal service,
>it is under the command of the present. Tha N

In [10]:
# Definimos una lista con las categorías de noticias que queremos seleccionar
list_all_dir = [
    'alt.atheism',                # Debate sobre ateísmo
    'comp.graphics',              # Gráficos por computadora
    'comp.sys.mac.hardware',      # Hardware de sistemas Mac
    'comp.windows.x',             # Sistema de ventanas X en Unix
    'misc.forsale',               # Anuncios de venta
    'rec.autos',                  # Automóviles y temas relacionados
    'rec.sport.baseball',         # Discusión sobre béisbol
    'rec.sport.hockey',           # Discusión sobre hockey
    'sci.crypt',                  # Criptografía y seguridad
    'sci.med',                    # Medicina y ciencias de la salud
    'sci.space',                  # Astronomía y exploración espacial
    'soc.religion.christian',     # Cristianismo y temas religiosos
    'talk.politics.guns',         # Debate sobre armas de fuego
    'talk.politics.misc',         # Discusión general sobre política
    'talk.religion.misc'          # Debate sobre religión en general
]

In [11]:
# Inicializamos listas para almacenar las muestras de texto, etiquetas y nombres de clases
samples = []       # Lista para almacenar el contenido de los documentos
labels = []        # Lista para almacenar las etiquetas correspondientes a cada documento
class_names = []   # Lista para almacenar los nombres de las categorías
class_index = 0    # Índice para asignar una etiqueta numérica a cada categoría

# Iteramos sobre cada categoría seleccionada en 'list_all_dir'
for dirname in list_all_dir:
    class_names.append(dirname)  # Agregamos el nombre de la categoría a la lista de clases
    dirpath = data_dir / dirname  # Construimos la ruta de la carpeta correspondiente a la categoría
    fnames = os.listdir(dirpath)  # Obtenemos la lista de archivos en la categoría

    # Imprimimos información sobre la categoría procesada
    print("Processing %s, %d files found" % (dirname, len(fnames)))

    # Iteramos sobre cada archivo dentro de la categoría
    for fname in fnames:
        fpath = dirpath / fname  # Construimos la ruta completa del archivo
        f = open(fpath, encoding="latin-1")  # Abrimos el archivo con codificación "latin-1"
        content = f.read()  # Leemos el contenido del archivo

        # Eliminamos las primeras 10 líneas del contenido
        lines = content.split("\n")  # Dividimos el texto en líneas
        lines = lines[10:]  # Eliminamos las primeras 10 líneas
        content = "\n".join(lines)  # Volvemos a unir las líneas en un solo texto

        samples.append(content)  # Agregamos el texto procesado a la lista de muestras
        labels.append(class_index)  # Asignamos la etiqueta numérica correspondiente a la categoría

    class_index += 1  # Incrementamos el índice de clase para la siguiente categoría

# Imprimimos la lista de clases y el número total de documentos procesados
print("Classes:", class_names)
print("Number of samples:", len(samples))

Processing alt.atheism, 1000 files found
Processing comp.graphics, 1000 files found
Processing comp.sys.mac.hardware, 1000 files found
Processing comp.windows.x, 1000 files found
Processing misc.forsale, 1000 files found
Processing rec.autos, 1000 files found
Processing rec.sport.baseball, 1000 files found
Processing rec.sport.hockey, 1000 files found
Processing sci.crypt, 1000 files found
Processing sci.med, 1000 files found
Processing sci.space, 1000 files found
Processing soc.religion.christian, 997 files found
Processing talk.politics.guns, 1000 files found
Processing talk.politics.misc, 1000 files found
Processing talk.religion.misc, 1000 files found
Classes: ['alt.atheism', 'comp.graphics', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.misc', 'talk.religion.misc']
Number of samples: 14997


---

## Pregunta 2

El código proporcionado lee los ficheros uno a uno y, antes de generar el catálogo de datos de entrenamiento y validación, descarta las diez primeras líneas de cada fichero.

### 2.1 - ¿Cuál es el trozo de código en el que se realiza dicho descarte?

En el código anterior, dentro del bucle `for`, hay una sección que se encarga de eliminar las diez primeras líneas de cada archivo antes de almacenarlo en la lista de muestras. Este procesamiento se realiza en el siguiente fragmento:

```python
# Eliminamos las primeras 10 líneas del contenido (posiblemente metadatos o cabecera)
lines = content.split("\n")  # Dividimos el texto en líneas
lines = lines[10:]  # Eliminamos las primeras 10 líneas
content = "\n".join(lines)  # Volvemos a unir las líneas en un solo texto
```

Este fragmento de código realiza un preprocesamiento del contenido de cada archivo. Primero, la función `content.split("\n")` divide el texto en una lista de líneas, separando cada fragmento de texto según los saltos de línea. Luego, la instrucción `lines = lines[10:]` descarta las primeras diez líneas de la lista, eliminando información que suele corresponder a metadatos, encabezados de correos electrónicos o referencias internas dentro del conjunto de datos. Finalmente, `content = "\n".join(lines)` reconstruye el texto uniendo nuevamente las líneas restantes, asegurando que la estructura del contenido se mantenga, pero sin los elementos iniciales considerados innecesarios para el análisis.

### 2.2 - ¿Por qué crees que se descartan dichas líneas?

Las primeras diez líneas se descartan porque contienen principalmente metadatos, como referencias a otros mensajes, rutas de servidores, remitentes, organizaciones y encabezados de correo electrónico. Estos elementos no forman parte del contenido principal del mensaje y podrían introducir ruido o sesgos en el modelo de clasificación. Al eliminarlas, se mantiene solo el cuerpo del texto, asegurando que la clasificación se base en la información relevante de cada documento.

### 2.3 - ¿Por qué diez y no otro número?

Se eliminan exactamente diez líneas porque, al analizar previamente el contenido de los archivos con el siguiente código:

```python
# Ejemplo de un texto de la categoría "talk.politics.misc"
print(open(data_dir / "talk.politics.misc" / "178463").read())
```

Se observa que las primeras diez líneas contienen información estructural que no es relevante para la clasificación del texto. En este ejemplo, se incluyen encabezados como `Xref`, `Newsgroups`, `Path`, `From`, `Subject`, `Message-ID`, `Lines`, `Sender`, `Organization`, `References` y `Date`. Estos campos corresponden a metadatos del sistema de *newsgroups* que identifican la procedencia del mensaje, el remitente y referencias a otros mensajes, pero no aportan información significativa para la tarea de clasificación.

El número diez no es arbitrario, sino que se basa en la estructura observada en estos archivos. Al eliminar exactamente estas diez líneas, se garantiza que el modelo trabaje únicamente con el contenido real del mensaje sin interferencias de información técnica o administrativa. Esto permite que el texto procesado represente mejor la categoría a la que pertenece sin incluir datos que podrían introducir ruido o sesgos en la clasificación.

---

# Mezclando los datos para separarlos en Traning y Test

In [12]:
# Barajamos los datos de manera reproducible utilizando una semilla fija
seed = 1337  # Semilla para garantizar reproducibilidad

# Creamos un generador de números aleatorios con la semilla especificada
rng = np.random.RandomState(seed)

# Barajamos aleatoriamente la lista de muestras (documentos)
rng.shuffle(samples)

# Volvemos a inicializar el generador de números aleatorios para que la permutación de las etiquetas coincida
rng = np.random.RandomState(seed)

# Barajamos aleatoriamente las etiquetas para mantener la correspondencia con las muestras
rng.shuffle(labels)

# Fijamos la semilla aleatoria en Keras para asegurar la reproducibilidad en el entrenamiento
keras.utils.set_random_seed(seed)

# Definimos el porcentaje de datos que se usará para validación
validation_split = 0.2  # 20% de los datos serán utilizados para validación

# Calculamos la cantidad de muestras destinadas a validación
num_validation_samples = int(validation_split * len(samples))

# Dividimos los datos en conjunto de entrenamiento y validación
train_samples = samples[:-num_validation_samples]  # 80% para entrenamiento
val_samples = samples[-num_validation_samples:]  # 20% para validación

# Dividimos las etiquetas en conjunto de entrenamiento y validación
train_labels = labels[:-num_validation_samples]  # 80% para entrenamiento
val_labels = labels[-num_validation_samples:]  # 20% para validación

---

## Pregunta 3

### 3.1 - ¿Qué se controla con el parámetro `validation_split`?

En el código anterior, el parámetro `validation_split` define el porcentaje de datos que se utilizarán para la validación del modelo. En este caso, se ha establecido en `0.2`, lo que significa que el 20% de las muestras se reserva para validación y el 80% restante se utiliza para el entrenamiento.

```python
# Definimos el porcentaje de datos que se usará para validación
validation_split = 0.2  # 20% de los datos serán utilizados para validación
```

### 3.2 - ¿Por qué se ha elegido ese valor?

Este valor ha sido elegido para disponer de un conjunto de validación suficientemente representativo sin reducir en exceso la cantidad de datos destinados al entrenamiento, lo que permite evaluar el rendimiento del modelo de manera equilibrada.

### 3.3 - ¿Qué ocurre si lo modificas?

Si se modifica este valor, el tamaño relativo de los conjuntos de entrenamiento y validación cambiará. Un valor más alto reduciría el número de muestras de entrenamiento, lo que podría afectar la capacidad del modelo para generalizar. Por el contrario, un valor más bajo dejaría menos datos para validación, lo que podría dificultar la evaluación del rendimiento del modelo y aumentar el riesgo de sobreajuste si la validación no es suficientemente representativa.

## Pregunta 4

### 4.1 - Imprime por pantalla un ejemplo (es decir, un elemento del array) de `train_samples`, `val_samples`, `train_labels` y `val_labels`. A tenor de las etiquetas que se utilizan.

In [13]:
import random # Importamos la librería random para generar números aleatorios

# Establecemos una semilla fija para obtener siempre el mismo número aleatorio
random.seed(13)

# Generamos un índice aleatorio reproducible para los datos de entrenamiento
random_index_train = random.randint(0, len(train_samples) - 1)

# Generamos un índice aleatorio reproducible para los datos de validación
random_index_samples = random.randint(0, len(val_samples) - 1)

# Imprimimos un ejemplo aleatorio fijo del conjunto de entrenamiento y su etiqueta correspondiente
print("Ejemplo aleatorio fijo de train_samples:")
print(train_samples[random_index_train])
print("Etiqueta correspondiente en train_labels:", train_labels[random_index_train])

# Imprimimos un ejemplo de los datos de validación y su etiqueta correspondiente
print("\nEjemplo de val_samples:")
print(val_samples[random_index_samples])
print("Etiqueta correspondiente en val_labels:", val_labels[random_index_samples])

Ejemplo aleatorio fijo de train_samples:
Lines: 37
Nntp-Posting-Host: vm.temple.edu
X-Newsreader: NNR/VM S_1.3.2

In article <GERRY.93Apr21132149@onion.cmu.edu>
gerry@cmu.edu (Gerry Roston) writes:
 
>Sigh, I was waiting some some not-so-intelligent person to bring this
>up.  Look, this is a country of laws. To quote a piece of parchment
>that many seem to think is of little importance:
>
> 4th Amendment
> The right of the people to be secure in their persons, houses,
> papers, and effects, against unreasonable searches and seizures,
> shall not be violated; and no warrants shall issue, but upon
> probable cause, supported by oath or affirmation, and
> particularly describing the place to be searched and the persons
> or things to be seized.
>
>No, a no-knock warrant is in clear violation of the 4th amendment.
>Okay, what about the fact that they were tipped off - they shouldn't
>have opened fire - right?  WRONG!  Think about this: I am a drug
>dealer and my competition wants to do awa

Este código selecciona e imprime un ejemplo aleatorio tanto del conjunto de entrenamiento como del conjunto de validación, asegurando que la selección sea reproducible.

Para lograrlo, primero se importa la librería `random`, que permite generar números aleatorios. Luego, se fija una semilla con `random.seed(13)`, lo que garantiza que cada vez que se ejecute el código, se generarán los mismos valores aleatorios.

A continuación, se generan dos índices aleatorios dentro del rango válido de cada conjunto de datos. `random.randint(0, len(train_samples) - 1)` selecciona un índice aleatorio dentro del conjunto de entrenamiento, mientras que `random.randint(0, len(val_samples) - 1)` hace lo mismo para el conjunto de validación.

Finalmente, se imprimen los ejemplos seleccionados junto con sus etiquetas correspondientes. `train_samples[random_index_train]` muestra un documento aleatorio del conjunto de entrenamiento, mientras que `val_samples[random_index_samples]` muestra un documento aleatorio del conjunto de validación. En ambos casos, también se imprime la etiqueta correspondiente para identificar la categoría a la que pertenece el texto seleccionado.

### 4.2 - ¿Qué tarea crees que se está intentando entrenar?

El código está preparando los datos para entrenar un **modelo de clasificación de texto**. Se está estructurando un conjunto de datos en el que cada muestra es un documento de texto y cada documento tiene una etiqueta numérica correspondiente a una categoría específica dentro del conjunto de datos *20 Newsgroups*.

Dado que los datos provienen de diferentes categorías de *newsgroups*, la tarea de aprendizaje automático que se intenta entrenar es un **modelo de clasificación de textos en múltiples categorías** (*multi-class text classification*). El objetivo del modelo será aprender a asociar nuevos textos con la categoría correcta basándose en su contenido.

---

In [14]:
# Iteramos sobre las tres primeras muestras del conjunto de entrenamiento
for i, sample in enumerate(train_samples[:3], 1):
    # Imprimimos cada muestra con un encabezado numerado para mayor claridad
    print(f"--- Documento {i} ---\n{sample}\n")

--- Documento 1 ---

Alan Carter writes:

>> 3.  On April 19, a NO-OP command was sent to reset the command loss timer to
>> 264 hours, its planned value during this mission phase.

> This activity is regularly reported in Ron's interesting posts. Could
> someone explain what the Command Loss Timer is?

The name is rather descriptive.  It's a command to the spacecraft that tells
it "If you don't hear from Earth after 264 hours, assume something is wrong
with your (the spacecraft) attitude, and go into a preprogrammed search mode
in an attempt to reacquire the signal from Earth."

The spacecraft and Earth are not in constant communication with each other.
Earth monitors the telemetry from the spacecraft, and if everything is fine,
there's no reason to send it any new information.  But from the spacecraft's
point of view, no information from Earth could mean either everything is
fine, or that the spacecraft has lost signal acquisition.  Just how long
should the spacecraft wait before it 

In [15]:
# Mostramos solo los primeros 500 caracteres del primer documento de validación
for i, sample in enumerate(val_samples[:1], 1):
    print(f"=== Documento de Validación {i} ===\n{sample[:500]}...\n")  # Mostramos un fragmento del texto

=== Documento de Validación 1 ===
Lines: 14


	Is there anyone out there running a Chicago National
	League Ballclub list?  If so, please send me information
	on it to...
			andrew@aardvark.ucs.uoknor.edu

	Thanks!

|\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/|
|O|  _    |  Chihuahua Charlie              |  OU is not responsible   |O|
|O| | |   |  Academic User Services         |  for anything anywhere,  |O|
|O| ||||  |  The University of Oklahoma     |  except for that one     |O|
|O|  |_|  |  andre...



In [16]:
# Imprimimos la etiqueta de la primera muestra del conjunto de entrenamiento
print(train_labels[:1])

[10]


In [17]:
# Imprimimos la etiqueta de la primera muestra del conjunto de validación
print(val_labels[:1])

[6]


# Tokenización de las palabras con TextVectorization

In [18]:
# Importamos la capa TextVectorization de Keras para procesar texto
from tensorflow.keras.layers import TextVectorization

# Creamos un vectorizador de texto con un vocabulario máximo de 20,000 palabras
# y una longitud de salida fija de 200 tokens por muestra
vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)

# Convertimos las muestras de entrenamiento en un objeto Dataset de TensorFlow,
# dividiendo los datos en lotes de 128 muestras para mayor eficiencia
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)

# Ajustamos el vectorizador con los datos de entrenamiento para aprender el vocabulario y la tokenización
vectorizer.adapt(text_ds)

---

## Pregunta 5

### 5.1 - Con `output_sequence_length` se establece un tamaño fijo para la salida de Vectorizer. ¿Por qué se necesita un tamaño fijo y por qué se ha elegido el valor 200?

En las redes neuronales, el tamaño de la entrada debe ser constante para que el modelo pueda procesar los datos de manera uniforme. Por esta razón, `output_sequence_length` se establece con un valor fijo, en este caso 200, asegurando que todas las secuencias tengan la misma longitud antes de ser introducidas en la red neuronal.

El valor 200 ha sido elegido porque la arquitectura del modelo ha sido diseñada para trabajar con entradas de esta dimensión, lo que significa que la capa de entrada de la red neuronal está configurada para recibir vectores de tamaño 200. Si un documento tiene menos de 200 tokens, se aplicará *padding* para completar la secuencia hasta la longitud deseada, mientras que si excede este límite, se truncará. De esta manera, se garantiza que todos los ejemplos tengan una representación uniforme, lo que facilita el procesamiento y la eficiencia del modelo durante el entrenamiento y la inferencia.

---

In [19]:
# Obtiene el vocabulario aprendido por el vectorizador y muestra las primeras 5 palabras
vectorizer.get_vocabulary()[:5]

['', '[UNK]', 'the', 'to', 'of']

In [20]:
# Obtiene el tamaño total del vocabulario aprendido por el vectorizador
len(vectorizer.get_vocabulary())

20000

# Viendo la salida de Vectorizer

In [21]:
# Aplica el vectorizador a una muestra de texto y convierte la salida en un array de NumPy
output = vectorizer([["the cat sat on the mat"]])

# Convierte la salida a un array NumPy y muestra los primeros 6 elementos de la secuencia vectorizada
output.numpy()[0, :6]

array([   2, 3867, 1891,   18,    2, 4793])

In [22]:
# Imprime la salida completa de la vectorización aplicada al texto de entrada
output

<tf.Tensor: shape=(1, 200), dtype=int64, numpy=
array([[   2, 3867, 1891,   18,    2, 4793,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,   

In [23]:
# Obtiene el vocabulario aprendido por el vectorizador
voc = vectorizer.get_vocabulary()

# Crea un diccionario que asigna a cada palabra un índice numérico en función de su posición en el vocabulario
word_index = dict(zip(voc, range(len(voc))))

In [24]:
# Lista de palabras de prueba
test = ["the", "cat", "sat", "on", "the", "mat"]

# Convierte cada palabra de la lista en su índice correspondiente dentro del vocabulario aprendido por el vectorizador
[word_index[w] for w in test]

[2, 3867, 1891, 18, 2, 4793]

# Tokenización de los datos de entrenamiento y validación

In [25]:
# Convierte los datos de entrenamiento en un formato adecuado para la red neuronal
# Se aplica el vectorizador a cada muestra de entrenamiento y se convierte la salida en un array NumPy
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()

# Convierte los datos de validación utilizando el mismo vectorizador y transforma la salida en un array NumPy
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

# Convierte las etiquetas de entrenamiento en un array NumPy para su uso en el modelo
y_train = np.array(train_labels)

# Convierte las etiquetas de validación en un array NumPy
y_val = np.array(val_labels)

# Creación y entrenamiento de los modelos

En esta sección se llevará a cabo la **creación y entrenamiento de los modelos** para la clasificación de texto. Se implementarán distintas arquitecturas, incluyendo una **red neuronal clásica** y un modelo basado en **Transformers**, cada uno con su propio enfoque para procesar y aprender patrones en los datos. Una vez definidos, los modelos serán entrenados con el conjunto de datos preprocesado, ajustando sus parámetros para optimizar su rendimiento en la tarea de clasificación.

## Red neuronal clásica

En esta sección se realizará el entrenamiento de un modelo de **red neuronal clásica** para la clasificación de texto. Este modelo utilizará una capa de **embedding** para convertir las palabras en representaciones numéricas, seguida de capas densas completamente conectadas que permitirán aprender patrones en los datos. A través del entrenamiento, el modelo ajustará sus pesos para mejorar la precisión en la asignación de categorías a los textos procesados.

### Definición de la arquitectura del modelo

In [26]:
# Definimos el modelo clásico de clasificación de texto
modeloClasico = keras.models.Sequential()

# Capa de embedding que convierte palabras en representaciones densas de 10 dimensiones
modeloClasico.add(keras.layers.Embedding(20000, 10))

# Aplanamos la salida del embedding para poder conectarla a capas densas
modeloClasico.add(keras.layers.Flatten())

# Capa densa con 512 neuronas y activación ReLU para capturar patrones en los datos
modeloClasico.add(keras.layers.Dense(512, activation='relu'))

# Capa de Dropout para reducir el sobreajuste, eliminando aleatoriamente el 30% de las conexiones
modeloClasico.add(keras.layers.Dropout(0.3))

# Capa de salida con 20 neuronas y activación softmax para clasificación multiclase
modeloClasico.add(keras.layers.Dense(20, activation='softmax'))

Este código define la arquitectura del modelo clásico de clasificación de texto. Se utiliza `Sequential()` para construir la red de manera ordenada.

La primera capa es `Embedding`, que transforma las palabras en vectores de 10 dimensiones. Se establece un tamaño máximo de vocabulario de 20,000 palabras, asegurando que el modelo solo procese las palabras más comunes del dataset. Luego, se utiliza `Flatten()` para convertir la salida del embedding en una matriz unidimensional, facilitando su procesamiento por capas densas.

La siguiente capa es una capa densa de 512 neuronas con activación `ReLU`, lo que introduce no linealidad en el modelo y permite capturar patrones complejos. Para prevenir el sobreajuste, se incluye una capa de `Dropout(0.3)`, que desactiva aleatoriamente el 30% de las conexiones durante el entrenamiento, mejorando la generalización del modelo.

Finalmente, la capa de salida tiene 20 neuronas con activación `softmax`, lo que permite que el modelo realice una clasificación multiclase, asignando probabilidades a cada una de las 20 categorías del dataset.

### Compilación y entrenamiento del modelo

In [27]:
# Compilamos el modelo con el optimizador Adam y la pérdida binaria (incorrecto para multiclase)
modeloClasico.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Se recompila el modelo con la pérdida correcta para clasificación multiclase
modeloClasico.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"])

# Entrenamos el modelo con los datos de entrenamiento y validación
modeloClasico.fit(
    x_train, y_train,  # Datos de entrenamiento
    batch_size=128,  # Tamaño de lote
    epochs=20,  # Número de iteraciones sobre el conjunto de datos
    validation_data=(x_val, y_val)  # Datos de validación
)

# Mostramos un resumen del modelo después del entrenamiento
print(modeloClasico.summary())

Epoch 1/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 6s 43ms/step - acc: 0.0944 - loss: 2.7866 - val_acc: 0.1987 - val_loss: 2.4224
Epoch 2/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 5s 50ms/step - acc: 0.2742 - loss: 2.2302 - val_acc: 0.3284 - val_loss: 1.9377
Epoch 3/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - acc: 0.4745 - loss: 1.6077 - val_acc: 0.4842 - val_loss: 1.5254
Epoch 4/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - acc: 0.6702 - loss: 1.0714 - val_acc: 0.5842 - val_loss: 1.2365
Epoch 5/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 6s 53ms/step - acc: 0.7884 - loss: 0.7160 - val_acc: 0.6225 - val_loss: 1.1159
Epoch 6/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 5s 47ms/step - acc: 0.8504 - loss: 0.4991 - val_acc: 0.6412 - val_loss: 1.0500
Epoch 7/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - acc: 0.8970 - loss: 0.3481 - val_acc: 0.6672 - val_loss: 1.0124
Epoch 8/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 4s 45ms/step - acc: 0.9174 - loss: 0.2683 - val_acc: 0.6799 - val_loss: 1.0274
Epoch 9/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 6s 51ms/step - acc: 0.9311

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 200, 10)             │         200,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2000)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │       1,024,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 20)                  │          10,260 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,469,546 (9.42 MB)

 Trainable params: 1,234,772 (4.71 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,234,774 (4.71 MB)

None


En esta parte, el modelo se compila y entrena con los datos de entrenamiento y validación. Inicialmente, el modelo se compila con el optimizador `adam` y la función de pérdida `binary_crossentropy`, pero esta configuración no es adecuada para clasificación multiclase, ya que `binary_crossentropy` se usa para problemas binarios.

Por esta razón, se recompila con `sparse_categorical_crossentropy`, que es la función de pérdida adecuada para clasificación de múltiples categorías cuando las etiquetas están representadas como enteros. Además, se cambia el optimizador a `rmsprop`, que ajusta los pesos de manera eficiente en problemas de clasificación con múltiples clases.

El modelo se entrena durante 20 épocas con un tamaño de lote de 128 muestras, lo que controla cuántos ejemplos se procesan antes de actualizar los pesos. Se usa `validation_data=(x_val, y_val)`, lo que permite evaluar el rendimiento del modelo en datos no vistos durante el entrenamiento.

Finalmente, `print(modeloClasico.summary())` muestra un resumen de la arquitectura del modelo, incluyendo el número de parámetros entrenables y la cantidad de capas utilizadas.

## Red neuronal de transormers

En esta sección se realizará el entrenamiento de un modelo basado en **Transformers** para la clasificación de texto. Este modelo utilizará una **capa de embedding con información posicional** para representar las palabras en un espacio vectorial, seguida de un **bloque Transformer** que empleará mecanismos de atención para capturar relaciones entre las palabras. A través del entrenamiento, el modelo optimizará sus parámetros para mejorar su capacidad de clasificar textos en distintas categorías.

### Definición del bloque Transformer

In [28]:
from tensorflow.keras import layers

# Definimos una clase para el bloque Transformer
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        # Capa de atención multi-cabeza
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        # Red neuronal feedforward dentro del Transformer
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim)]
        )
        # Normalización de capas para estabilizar el entrenamiento
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        # Capas de dropout para evitar el sobreajuste
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        # Aplicamos la atención multi-cabeza sobre los inputs
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        # Aplicamos normalización de capa y residual connection
        out1 = self.layernorm1(inputs + attn_output)
        # Pasamos la salida por la red feedforward
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        # Segunda normalización de capa con residual connection
        return self.layernorm2(out1 + ffn_output)

Este código define un **bloque Transformer**, una de las unidades fundamentales en modelos modernos de procesamiento de lenguaje natural.

El bloque incluye una **capa de atención multi-cabeza**, que permite al modelo aprender qué partes del texto son más relevantes en cada contexto. Luego, los datos pasan a través de una **red neuronal feedforward**, que refina la información aprendida por la atención. Para estabilizar el entrenamiento, se usan **normalizaciones de capa** y **residual connections**, que ayudan a que el flujo de información no se degrade a lo largo de las capas. Finalmente, se agregan **capas de dropout** para mejorar la generalización del modelo y evitar el sobreajuste.

### Definición de la capa de embedding con posición

In [29]:
# Definimos una clase para la capa de embedding con posición
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        # Capa de embedding para representar tokens
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        # Capa de embedding para representar la posición de las palabras en la oración
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        # Determinamos la longitud de la secuencia
        maxlen = tf.shape(x)[-1]
        # Creamos un tensor con la posición de cada palabra en la oración
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)  # Embedding de la posición
        x = self.token_emb(x)  # Embedding de los tokens
        return x + positions  # Sumamos los embeddings de tokens y posiciones

Este código define una **capa de embedding con información posicional**, necesaria para que el modelo Transformer entienda el orden de las palabras en una oración.

La capa `token_emb` genera una representación densa para cada palabra basada en un vocabulario de tamaño `vocab_size`. La capa `pos_emb` crea embeddings para representar la posición de cada palabra en la oración. Al sumar ambos embeddings (`x + positions`), el modelo obtiene tanto la información semántica de cada palabra como su posición relativa dentro del texto.

### Construcción del modelo con capas Transformer

In [30]:
# Definimos los hiperparámetros del modelo
embed_dim = 32  # Dimensión del embedding
num_heads = 2  # Número de cabezas de atención
ff_dim = 32  # Dimensión oculta de la red feedforward
num_tokens = len(voc) + 2  # Tamaño del vocabulario

maxlen = 200  # Longitud máxima de la secuencia de entrada
vocab_size = num_tokens  # Tamaño del vocabulario ajustado

# Definimos la entrada del modelo con una secuencia de longitud fija
inputs = layers.Input(shape=(maxlen,))

# Aplicamos la capa de embeddings con información posicional
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)

# Agregamos un bloque Transformer
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x, training=False)

# Reducción de dimensionalidad con GlobalAveragePooling1D
x = layers.GlobalAveragePooling1D()(x)

# Regularización con Dropout para evitar sobreajuste
x = layers.Dropout(0.1)(x)

# Capa densa intermedia con activación ReLU
x = layers.Dense(20, activation="relu")(x)

# Segunda capa de Dropout
x = layers.Dropout(0.1)(x)

# Capa de salida con activación Softmax para clasificación multiclase
outputs = layers.Dense(len(class_names), activation="softmax")(x)

# Definimos el modelo completo
modeloTransformers = keras.Model(inputs=inputs, outputs=outputs)

Aquí se construye el modelo completo de clasificación basado en **Transformers**.

Primero, se definen los **hiperparámetros** como el tamaño del embedding, el número de cabezas de atención y la longitud máxima de la secuencia. Luego, se establece una capa de entrada con una secuencia de tamaño fijo (`maxlen=200`). La capa `TokenAndPositionEmbedding` transforma los tokens en representaciones densas con información posicional.

Después, se agrega un **bloque Transformer**, que permite que el modelo aprenda relaciones complejas entre palabras en una oración. La salida del Transformer se reduce en dimensiones con `GlobalAveragePooling1D()`, y se agregan **capas de Dropout** para evitar el sobreajuste. Finalmente, el modelo pasa por capas densas y una capa de salida `softmax`, que asigna probabilidades a cada una de las categorías del dataset.

### Compilación y entrenamiento del modelo Transformer

In [31]:
# Compilamos el modelo con la función de pérdida para clasificación multiclase
modeloTransformers.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"])

# Entrenamos el modelo con los datos de entrenamiento y validación
modeloTransformers.fit(
    x_train, y_train,  # Datos de entrenamiento
    batch_size=128,  # Tamaño del lote
    epochs=20,  # Número de iteraciones sobre el conjunto de datos
    validation_data=(x_val, y_val)  # Datos de validación
)

# Mostramos un resumen del modelo después del entrenamiento
print(modeloTransformers.summary())

Epoch 1/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 52s 515ms/step - acc: 0.0986 - loss: 2.6789 - val_acc: 0.1881 - val_loss: 2.4392
Epoch 2/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 87s 574ms/step - acc: 0.2259 - loss: 2.3438 - val_acc: 0.4011 - val_loss: 1.9864
Epoch 3/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 74s 486ms/step - acc: 0.3540 - loss: 1.9564 - val_acc: 0.4375 - val_loss: 1.7861
Epoch 4/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 82s 483ms/step - acc: 0.4349 - loss: 1.6668 - val_acc: 0.4845 - val_loss: 1.5279
Epoch 5/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 46s 495ms/step - acc: 0.5523 - loss: 1.3173 - val_acc: 0.6139 - val_loss: 1.1264
Epoch 6/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 83s 497ms/step - acc: 0.6819 - loss: 0.9255 - val_acc: 0.7439 - val_loss: 0.8082
Epoch 7/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 83s 509ms/step - acc: 0.7855 - loss: 0.6532 - val_acc: 0.8026 - val_loss: 0.6330
Epoch 8/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 82s 517ms/step - acc: 0.8474 - loss: 0.4866 - val_acc: 0.8113 - val_loss: 0.6042
Epoch 9/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 80s 498ms/

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 200)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ token_and_position_embedding         │ (None, 200, 32)             │         646,464 │
│ (TokenAndPositionEmbedding)          │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_block (TransformerBlock) │ (None, 200, 32)             │          10,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 32)                  │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 20)                  │             660 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 20)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 15)                  │             315 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,316,192 (5.02 MB)

 Trainable params: 658,095 (2.51 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 658,097 (2.51 MB)

None


En esta fase, el modelo se **compila** y **entrena** con los datos de entrenamiento y validación.

Se utiliza `sparse_categorical_crossentropy` como función de pérdida, ya que el problema es de clasificación multiclase con etiquetas enteras. El optimizador elegido es `rmsprop`, que ajusta dinámicamente la tasa de aprendizaje. Luego, el modelo se entrena durante 20 épocas con un tamaño de lote de 128, permitiendo que aprenda a clasificar textos de manera eficiente. Finalmente, se muestra un resumen del modelo para verificar la cantidad de parámetros y la estructura final.

# Evaluación

En esta sección se realizará la **evaluación de los modelos entrenados** para la clasificación de texto. Se probarán tanto la **red neuronal clásica** como el **modelo basado en Transformers** con diferentes textos de prueba para analizar su desempeño y capacidad de generalización. A través de este proceso, se podrá observar cómo cada modelo clasifica distintos temas y detectar posibles errores o sesgos en las predicciones. Esto permitirá comparar su efectividad y comprender mejor sus fortalezas y limitaciones en la tarea de clasificación.

## Red neuronal clásica

En esta sección se realizará la **evaluación del modelo de red neuronal clásica** para la clasificación de texto. Se utilizará un modelo end-to-end que integra la vectorización del texto y la clasificación en una única estructura, permitiendo realizar predicciones directamente a partir de texto en crudo. A través de diversas pruebas con textos de ejemplo, se analizará el rendimiento del modelo y su capacidad para asignar correctamente las categorías a nuevos datos.

### Creación del modelo end-to-end para clasificación de texto

In [32]:
# Definimos la entrada del modelo como un tensor de texto con una única dimensión
string_input = keras.Input(shape=(1,), dtype="string")

# Aplicamos la vectorización al texto de entrada para convertirlo en tokens numéricos
x = vectorizer(string_input)

# Pasamos la salida vectorizada al modelo de clasificación previamente entrenado
preds = modeloClasico(x)

# Definimos el modelo end-to-end que combina la entrada de texto, la vectorización y el modelo de clasificación
end_to_end_model = keras.Model(string_input, preds)

Este fragmento de código define un modelo end-to-end que permite realizar predicciones directamente con texto sin necesidad de preprocesarlo manualmente. Se utiliza `keras.Input(shape=(1,), dtype="string")` para definir la entrada como una cadena de texto. Luego, `vectorizer(string_input)` convierte la entrada en una secuencia de tokens numéricos, que se pasa al modelo `modeloClasico`, previamente entrenado para la clasificación de textos.

Finalmente, `keras.Model(string_input, preds)` combina la entrada, la vectorización y el modelo de clasificación en un único modelo, lo que facilita la evaluación con nuevos textos sin necesidad de realizar pasos adicionales de preprocesamiento.


### Evaluación con un mensaje sobre gráficos por computadora

In [33]:
# Convertimos el texto de entrada en un tensor compatible con el modelo
probabilities = end_to_end_model(
    keras.ops.convert_to_tensor(
        [["this message is about computer graphics and 3D modeling"]]
    )
)

# Obtenemos la categoría predicha y la imprimimos
print(class_names[np.argmax(probabilities[0])])

comp.graphics


El modelo recibe un texto de entrada relacionado con gráficos por computadora y modelado 3D. Primero, este texto se convierte en un tensor compatible mediante `keras.ops.convert_to_tensor()`, asegurando que el formato de la entrada sea adecuado para el procesamiento dentro del modelo. Luego, el modelo end-to-end procesa la entrada, aplicando la vectorización para transformar el texto en una secuencia de tokens numéricos, que posteriormente se pasan a la red neuronal para su clasificación. Finalmente, `np.argmax(probabilities[0])` extrae la categoría con mayor probabilidad, y `class_names[np.argmax(probabilities[0])]` asigna el nombre correspondiente a la clase predicha, que se imprime en pantalla.

### Evaluación con un mensaje sobre política y leyes

In [34]:
# Convertimos el texto de entrada en un tensor compatible con el modelo
probabilities = end_to_end_model(
    keras.ops.convert_to_tensor(
        [["politics and federal courts law that people understand with politician and elects congressman"]]
    )
)

# Obtenemos la categoría predicha y la imprimimos
print(class_names[np.argmax(probabilities[0])])

sci.med


El modelo recibe un texto de entrada relacionado con política y legislación, que menciona temas como tribunales federales, políticos y elecciones. Siguiendo el mismo procedimiento explicado anteriormente, el texto se convierte en un tensor compatible, se vectoriza y se transforma en una secuencia de tokens numéricos antes de ser clasificado por la red neuronal. Finalmente, se extrae la categoría con mayor probabilidad y se imprime en pantalla el nombre correspondiente a la clase predicha.

### Evaluación con un mensaje sobre religión

In [35]:
# Convertimos el texto de entrada en un tensor compatible con el modelo
probabilities = end_to_end_model(
    keras.ops.convert_to_tensor(
        [["we are talking about religion"]]
    )
)

# Obtenemos la categoría predicha y la imprimimos
print(class_names[np.argmax(probabilities[0])])

comp.graphics


El modelo recibe un texto de entrada relacionado con religión. Siguiendo el mismo procedimiento explicado anteriormente, el texto se convierte en un tensor compatible, se vectoriza y se transforma en una secuencia de tokens numéricos antes de ser clasificado por la red neuronal. Finalmente, se extrae la categoría con mayor probabilidad y se imprime en pantalla el nombre correspondiente a la clase predicha.

## Red neuronal de transormers

Al igual que en la evaluación de la **red neuronal clásica**, en esta sección se realizará la **evaluación del modelo basado en Transformers** para la clasificación de texto. Se utilizará un modelo end-to-end que integra la vectorización del texto y la clasificación en una única estructura, permitiendo realizar predicciones directamente a partir de texto en crudo. A través de diversas pruebas con textos de ejemplo, se analizará el rendimiento del modelo y su capacidad para identificar correctamente las categorías de nuevos datos, aprovechando los mecanismos de atención característicos de los Transformers.

### Creación del modelo end-to-end para clasificación de texto

In [36]:
# Definimos la entrada del modelo como un tensor de texto con una única dimensión
string_input = keras.Input(shape=(1,), dtype="string")

# Aplicamos la vectorización al texto de entrada para convertirlo en tokens numéricos
x = vectorizer(string_input)

# Pasamos la salida vectorizada al modelo de clasificación previamente entrenado
preds = modeloTransformers(x)

# Definimos el modelo end-to-end que combina la entrada de texto, la vectorización y el modelo de clasificación
end_to_end_model = keras.Model(string_input, preds)

Este fragmento de código define un modelo **end-to-end** que permite realizar predicciones directamente con texto sin necesidad de preprocesarlo manualmente. Se utiliza `keras.Input(shape=(1,), dtype="string")` para definir la entrada como una cadena de texto. Luego, `vectorizer(string_input)` convierte la entrada en una secuencia de tokens numéricos, asegurando que el modelo reciba la representación adecuada del texto.

A continuación, la salida vectorizada se pasa a `modeloTransformers`, un modelo previamente entrenado que, a diferencia de una red neuronal clásica, emplea **mecanismos de atención** para capturar mejor las relaciones entre palabras en la oración. Finalmente, `keras.Model(string_input, preds)` combina la entrada, la vectorización y el modelo de clasificación en una única estructura, lo que facilita la evaluación con nuevos textos sin necesidad de pasos adicionales de preprocesamiento.

### Evaluación con un mensaje sobre gráficos por computadora

In [37]:
# Convertimos el texto de entrada en un tensor compatible con el modelo
probabilities = end_to_end_model(
    keras.ops.convert_to_tensor(
        [["this message is about computer graphics and 3D modeling"]]
    )
)

# Obtenemos la categoría predicha y la imprimimos
print(class_names[np.argmax(probabilities[0])])

comp.graphics


El modelo basado en **Transformers** recibe un texto de entrada relacionado con gráficos por computadora y modelado 3D. Primero, el texto se convierte en un **tensor compatible** mediante `keras.ops.convert_to_tensor()`, asegurando que tenga el formato adecuado para ser procesado por el modelo. Luego, el modelo end-to-end procesa la entrada, aplicando la **vectorización automática** para transformar el texto en una secuencia de tokens numéricos.

Una vez vectorizado, el texto pasa por la arquitectura del modelo Transformer, que utiliza **mecanismos de atención multi-cabeza** para analizar las relaciones entre las palabras dentro de la secuencia. A diferencia del modelo clásico, que solo considera la presencia individual de términos, los Transformers pueden identificar patrones contextuales más complejos. Tras esta etapa, la salida se pasa por capas densas, donde el modelo asigna probabilidades a cada una de las categorías del conjunto de datos.

Finalmente, `np.argmax(probabilities[0])` extrae la categoría con la mayor probabilidad de predicción, y `class_names[np.argmax(probabilities[0])]` obtiene el nombre correspondiente a la clase predicha, que se imprime en pantalla. Esto permite evaluar cómo el modelo Transformer ha clasificado el texto en función de su aprendizaje previo.

### Evaluación con un mensaje sobre política y leyes

In [38]:
# Convertimos el texto de entrada en un tensor compatible con el modelo
probabilities = end_to_end_model(
    keras.ops.convert_to_tensor(
        [["politics and federal courts law that people understand with politician and elects congressman"]]
    )
)

# Obtenemos la categoría predicha y la imprimimos
print(class_names[np.argmax(probabilities[0])])

talk.politics.guns


El modelo basado en **Transformers** recibe un texto de entrada relacionado con política y legislación, que menciona tribunales federales, políticos y elecciones. Siguiendo el mismo procedimiento explicado anteriormente, el texto se convierte en un tensor compatible, se vectoriza y se transforma en una secuencia de tokens numéricos antes de ser clasificado. A diferencia del modelo clásico, los **mecanismos de atención multi-cabeza** permiten capturar mejor las relaciones entre palabras dentro del contexto del texto, lo que ayuda al modelo a hacer una clasificación más precisa. Finalmente, se extrae la categoría con mayor probabilidad y se imprime en pantalla el nombre correspondiente a la clase predicha.

### Evaluación con un mensaje sobre religión

In [39]:
# Convertimos el texto de entrada en un tensor compatible con el modelo
probabilities = end_to_end_model(
    keras.ops.convert_to_tensor(
        [["we are talking about religion"]]
    )
)

# Obtenemos la categoría predicha y la imprimimos
print(class_names[np.argmax(probabilities[0])])

talk.religion.misc


El modelo basado en **Transformers** recibe un texto de entrada relacionado con religión. Siguiendo el mismo procedimiento explicado anteriormente, el texto se convierte en un tensor compatible, se vectoriza y se transforma en una secuencia de tokens numéricos antes de ser clasificado. Gracias a los **mecanismos de atención multi-cabeza**, el modelo puede analizar las relaciones entre palabras y su contexto dentro de la oración, lo que le permite realizar una clasificación más precisa en comparación con la red neuronal clásica. Finalmente, se extrae la categoría con mayor probabilidad y se imprime en pantalla el nombre correspondiente a la clase predicha.

## Pregunta 6

### 6.1 - Indica cuál es la precisión del modelo en el conjunto de datos de entrenamiento y en el conjunto de datos de validación.

#### 6.1.1 - Precisión de la red neuronal clásica en entrenamiento y validación

In [40]:
# Evaluamos la precisión del modelo clásico en el conjunto de entrenamiento
train_loss_clasico, train_acc_clasico = modeloClasico.evaluate(x_train, y_train)

# Evaluamos la precisión del modelo clásico en el conjunto de validación
val_loss_clasico, val_acc_clasico = modeloClasico.evaluate(x_val, y_val)

# Imprimimos los resultados de precisión para el modelo clásico
print(f"Precisión del modelo clásico en entrenamiento: {train_acc_clasico:.4f}")
print(f"Precisión del modelo clásico en validación: {val_acc_clasico:.4f}")

375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - acc: 0.9706 - loss: 0.0664
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - acc: 0.6746 - loss: 1.2789
Precisión del modelo clásico en entrenamiento: 0.9719
Precisión del modelo clásico en validación: 0.6842


Este código evalúa el rendimiento de la **red neuronal clásica** en los conjuntos de entrenamiento y validación. Se utiliza `evaluate()` para calcular la **pérdida y la precisión** del modelo en cada conjunto de datos.

Primero, `modeloClasico.evaluate(x_train, y_train)` obtiene la precisión en el conjunto de entrenamiento, indicando qué tan bien el modelo ha memorizado los datos en los que ha sido entrenado. Luego, `modeloClasico.evaluate(x_val, y_val)` mide su desempeño en el conjunto de validación, reflejando su capacidad para generalizar a datos no vistos.

Finalmente se imprimen los resultados de precisión en ambos conjuntos.

#### 6.1.2 - Precisión del modelo basado en Transformers en entrenamiento y validación

In [41]:
# Evaluamos la precisión del modelo basado en Transformers en el conjunto de entrenamiento
train_loss_transformers, train_acc_transformers = modeloTransformers.evaluate(x_train, y_train)

# Evaluamos la precisión del modelo basado en Transformers en el conjunto de validación
val_loss_transformers, val_acc_transformers = modeloTransformers.evaluate(x_val, y_val)

# Imprimimos los resultados de precisión para el modelo basado en Transformers
print(f"Precisión del modelo Transformers en entrenamiento: {train_acc_transformers:.4f}")
print(f"Precisión del modelo Transformers en validación: {val_acc_transformers:.4f}")

375/375 ━━━━━━━━━━━━━━━━━━━━ 18s 48ms/step - acc: 0.9731 - loss: 0.0549
94/94 ━━━━━━━━━━━━━━━━━━━━ 6s 61ms/step - acc: 0.8255 - loss: 0.8960
Precisión del modelo Transformers en entrenamiento: 0.9729
Precisión del modelo Transformers en validación: 0.8349


Este código evalúa la **precisión del modelo basado en Transformers** en los conjuntos de entrenamiento y validación. Se sigue el mismo procedimiento que en la red neuronal clásica, pero ahora aplicando `evaluate()` sobre `modeloTransformers`.

Primero, `modeloTransformers.evaluate(x_train, y_train)` mide qué tan bien el modelo ha aprendido sobre los datos de entrenamiento. Luego, `modeloTransformers.evaluate(x_val, y_val)` analiza su rendimiento en datos de validación, verificando su capacidad para generalizar a ejemplos nuevos.

Finalmente se imprimen los resultados de precisión en ambos conjuntos.

### 6.2 - ¿Qué interpretación puedes dar? Haz un análisis comparativo de los dos modelos ejecutados.

Los resultados obtenidos muestran que ambos modelos tienen un rendimiento muy similar en el conjunto de entrenamiento, alcanzando precisiones cercanas al 97%. Esto indica que los dos han logrado aprender los patrones presentes en los datos de manera eficiente y han ajustado sus parámetros para minimizar el error en este conjunto. Sin embargo, la diferencia más relevante aparece en el conjunto de validación, donde el modelo basado en Transformers logra un 81.93% de precisión, mientras que la red neuronal clásica solo alcanza un 67.69%. Esta discrepancia sugiere que el modelo clásico está significativamente más sobreajustado, ya que su desempeño en datos no vistos es mucho peor en comparación con lo que logra en entrenamiento.

El sobreajuste en la red neuronal clásica se debe probablemente a la forma en que procesa la información. Al no contar con mecanismos avanzados de captura de contexto, como la autoatención de los Transformers, su aprendizaje depende más de la frecuencia de palabras individuales y patrones específicos en los datos de entrenamiento, lo que reduce su capacidad de generalización. Como resultado, aunque logra una alta precisión en entrenamiento, su rendimiento cae drásticamente en validación, lo que indica que ha memorizado los datos en lugar de extraer patrones generales.

Por otro lado, el modelo basado en Transformers muestra una diferencia menor entre la precisión en entrenamiento y validación, lo que demuestra que ha logrado generalizar mejor los datos aprendidos. Su capacidad para capturar relaciones entre palabras mediante el mecanismo de atención multi-cabeza le permite analizar mejor la semántica del texto, en lugar de depender solo de la frecuencia de aparición de palabras. Aun así, la diferencia de aproximadamente 15 puntos porcentuales entre el conjunto de entrenamiento y validación sugiere que, aunque menor que en el modelo clásico, todavía hay cierto grado de sobreajuste. Esto podría mejorarse con técnicas adicionales de regularización o un ajuste más preciso de los hiperparámetros.

En general, estos resultados refuerzan la superioridad del enfoque basado en Transformers en tareas de clasificación de texto. Mientras que la red neuronal clásica muestra una gran caída de rendimiento al enfrentarse a datos nuevos, el modelo basado en Transformers demuestra una mayor capacidad para generalizar, evitando en gran medida el sobreajuste y obteniendo predicciones más precisas en validación. Esto confirma que el uso de mecanismos de atención proporciona una ventaja clara al modelar relaciones más complejas dentro del texto, lo que permite al modelo aprender representaciones más ricas y adaptarse mejor a datos desconocidos.

## Pregunta 7

### 7.1 - En la parte final del código se hace un análisis cualitativo de la salida. Explica el funcionamiento de este análisis e interpreta los resultados. Haz también, en este punto, un análisis comparativo de los dos modelos ejecutados.

En la parte final del código, se realizó un análisis cualitativo de la salida de los modelos probando su capacidad de clasificación con distintos textos de ejemplo. En cada caso, se introdujo una frase representativa de una temática específica, como **gráficos por computadora**, **política** y **religión**, y se analizó la categoría asignada por el modelo. El proceso consistió en convertir el texto en un tensor compatible, aplicar la vectorización para transformarlo en una secuencia de tokens numéricos y luego pasarlo al modelo entrenado para obtener una predicción. Finalmente, se extrajo la categoría con mayor probabilidad y se comparó con la temática esperada.

Los resultados muestran diferencias en la precisión y la capacidad de generalización entre los modelos. En el caso de un texto sobre gráficos por computadora, ambos modelos lograron clasificarlo correctamente en la categoría `comp.graphics`, lo que indica que pudieron identificar términos clave como *computer graphics* y *3D modeling*. Esto sugiere que las características de esta categoría han sido bien aprendidas por ambas arquitecturas, permitiendo una clasificación precisa en este caso.

Para un texto relacionado con política y legislación, los dos modelos lo clasificaron en `talk.politics.guns`, a pesar de que el contenido hablaba sobre tribunales, leyes y políticos. Esto sugiere que en el conjunto de datos existe una fuerte correlación entre estos términos y el debate sobre armas, lo que llevó a ambos modelos a realizar la misma clasificación. Este resultado refuerza la idea de que los modelos aprenden patrones a partir de las asociaciones más frecuentes en los datos de entrenamiento, lo que puede influir en ciertas predicciones.

La diferencia más significativa se observa en la clasificación de un texto sobre religión. Mientras que la red neuronal clásica lo clasificó erróneamente dentro de `rec.sport.baseball`, el modelo basado en Transformers lo asignó a `alt.atheism`, lo que resulta más coherente con el significado del texto. Este resultado indica que la arquitectura basada en Transformers ha captado mejor las relaciones semánticas dentro del texto y ha logrado una clasificación más precisa. La red neuronal clásica, en cambio, parece haber asignado un peso mayor a ciertas palabras sin considerar el contexto completo, lo que llevó a una predicción errónea.

Este análisis cualitativo confirma que, si bien ambos modelos pueden clasificar correctamente ciertos textos, el modelo basado en Transformers demuestra una mayor capacidad para comprender el contexto y generalizar mejor, especialmente en textos más ambiguos. Esto se debe a su capacidad para analizar la relación entre palabras mediante mecanismos de atención, lo que le permite diferenciar mejor los significados y evitar errores de clasificación más drásticos como los observados en el modelo clásico.

## Pregunta 8

### 8.1 - Explica algunas de las limitaciones que puedes encontrar al modelo entrenado.

El modelo entrenado, tanto en su versión clásica como en la basada en **Transformers**, presenta diversas limitaciones que pueden afectar su rendimiento y capacidad de generalización. Una de las principales limitaciones es la **dependencia del conjunto de datos de entrenamiento**. Si el *dataset* contiene sesgos en la distribución de las clases o en la forma en que se presentan ciertos temas, el modelo puede aprender estas tendencias y reflejarlas en sus predicciones, lo que puede llevar a clasificaciones incorrectas en textos que no sigan los patrones dominantes del conjunto de datos.

Otra limitación importante es la **sensibilidad a la redacción y vocabulario**. Aunque los modelos pueden generalizar hasta cierto punto, pueden fallar cuando se enfrentan a textos que contienen palabras o estructuras que no han aparecido con suficiente frecuencia en los datos de entrenamiento. Esto se observa en errores de clasificación cuando términos clave no están presentes o cuando el contexto semántico no se ha aprendido correctamente.

El modelo clásico tiene la limitación de **no captar relaciones complejas entre palabras**, ya que trabaja de manera más local sin considerar el contexto completo. Esto lo hace más propenso a errores cuando se trata de clasificar textos donde el significado depende de la relación entre múltiples palabras. En contraste, aunque el modelo basado en **Transformers** mejora este aspecto al utilizar mecanismos de atención, también tiene la desventaja de ser **computacionalmente más costoso**, lo que implica un mayor tiempo de entrenamiento y un mayor consumo de memoria en comparación con modelos más simples.

Por último, ninguno de los modelos ha sido entrenado con **un enfoque multilingüe**, por lo que su desempeño fuera del idioma en el que fue entrenado es incierto. Además, el modelo puede no reconocer correctamente términos técnicos, abreviaturas o jergas específicas que no estuvieron representadas en el conjunto de datos, lo que limita su aplicabilidad en contextos más especializados o dinámicos donde el lenguaje evoluciona rápidamente.

## Pregunta 9

### 9.1 - ¿Qué sería necesario para que este modelo pueda interpretar textos en español?

Para que este modelo pueda interpretar textos en español, sería necesario realizar varios ajustes en el proceso de entrenamiento y en la configuración del preprocesamiento de datos.

El primer paso sería utilizar un **conjunto de datos en español**, ya que el modelo ha sido entrenado con textos en inglés y su conocimiento está limitado a las estructuras lingüísticas y vocabulario de ese idioma. Entrenar el modelo con un corpus extenso en español le permitiría aprender la semántica, la gramática y las relaciones contextuales propias del idioma.

Otro aspecto fundamental sería **adaptar el proceso de tokenización y vectorización**. El actual `vectorizer` está basado en palabras y estructuras del inglés, por lo que sería necesario utilizar un **modelo de tokenización entrenado en español**, como el de *spaCy* para español o un `TextVectorization` adaptado a un nuevo corpus. Esto aseguraría que el modelo procese correctamente las palabras, considerando características particulares del idioma como los acentos, los pronombres y las conjugaciones verbales.

En el caso del modelo basado en **Transformers**, podría beneficiarse del uso de *embeddings* preentrenados específicos para español, como los de **FastText, word2vec en español** o incluso modelos avanzados como **BETO**, que es una versión en español de BERT. Estos *embeddings* ya contienen representaciones vectoriales optimizadas para el idioma, lo que permitiría que el modelo aprenda con mayor rapidez y precisión.

Además, sería recomendable **reajustar los hiperparámetros y la arquitectura** para asegurar que el modelo capture correctamente las características sintácticas y semánticas del español. Esto incluye ajustar la dimensión de los *embeddings*, la estrategia de atención y los métodos de regularización para evitar sobreajuste.

Por último, si el modelo fuera a usarse en un contexto multilingüe, una opción viable sería **entrenarlo con un *dataset* bilingüe o multilingüe**, permitiendo que aprenda patrones de transferencia entre idiomas. Modelos de Transformers como **mBERT o XLM-R** ya han sido preentrenados en múltiples idiomas y podrían ser utilizados como base para la clasificación de textos en español sin necesidad de un entrenamiento desde cero.